<a href="https://colab.research.google.com/github/rieko0903/NaturalLanguageProcessing01/blob/test02/natural_language_processing01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# pip

In [ ]:
!pip install janome

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.7 MB 1.2 MB/s 


# import

In [68]:
import tensorflow as tf
import glob
import logging
import os
import re
import urllib
import warnings
import zipfile

import gensim
import numpy as np
import tensorflow as tf

from bs4 import BeautifulSoup
from urllib import request
from gensim.models.word2vec import Word2Vec
from janome.tokenizer import Tokenizer

In [69]:
URL = 'https://www.aozora.gr.jp/cards/000074/files/427_19793.html'
t = Tokenizer(wakati=True)

# 学習の準備

## 文章のスクレイピング
* xtml形式の表示からclass=main_textをスクレイピングする
* 本文以外の要素を削除する(brタグ等)

In [70]:
def load_aozora_text(url):
    response = request.urlopen(url)
    soup = BeautifulSoup(response)
    response.close()
    text = soup.find('div', class_='main_text')
    tags_to_delete = text.find_all(['rp', 'rt'])
    for tag in tags_to_delete:
        tag.decompose()
    text = text.get_text()
    text = text.replace('\r', '').replace('\n', '').replace('\u3000', '')
    return text

## 文章を扱いやすくするための事前処理

In [95]:
def preprocess(text):
    pass

## インプット文字列とターゲット文字列を１ずらす

In [82]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

In [92]:
class AozoraText:
  def __init__(self, url):
    self.text = load_aozora_text(URL)
    self.wakati, self.word2idx, self.idx2word, self.corpus = preprocess(self.text)

  def make_writer_dataset(self, batch_size, seq_length):
    data = tf.data.Dataset.from_tensor_slices(self.corpus)
    sequences = data.batch(seq_length+1, drop_remainder=True)
    dataset = sequences.map(split_input_target)
    return dataset.shuffle(buffer_size=10000).batch(batch_size=batch_size, drop_remainder=True)

In [94]:
if __name__ == "__main__":
    aozora = AozoraText(URL)

AttributeError: ignored